In [ ]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/FastaiNotebook_00_load_data")' FastaiNotebook_00_load_data

Installing packages:
	.package(path: "/home/clattner/fastai_docs/dev_swift/FastaiNotebook_00_load_data")
		FastaiNotebook_00_load_data
With SwiftPM flags: []
Working in: /tmp/tmprgs_iy0p
/home/clattner/swift/usr/bin/swift-build: /home/clattner/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/clattner/swift/usr/lib/swift/linux/libFoundation.so)
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 1.93s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
/home/clattner/swift/usr/bin/swiftc: /home/clattner/anaconda3/envs/swift/lib/libuuid.so.1: no version information available (required by /home/clattner/swift/usr/bin/swiftc)
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'Path' (9 sources)
/home/clattner/swift/usr/bin/swiftc: /home/clattner

: 

In [ ]:
//export
import Path
import TensorFlow

In [ ]:
import FastaiNotebook_00_load_data

## Load the data and get some Tensors to play with

In [ ]:
// loadMNIST is defined in workbook 00_load_data.
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath, flat: true)

var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784)
var bias = Tensor<Float>(zeros: [10])
print(bias)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# Building Matmul

Ok, now that we know how floating point types and arrays work, we can finally build our own matmul from scratch, using a few loops.  We will take the two input matrices as flattened arrays so we can show manual indexing into them:

In [ ]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmul(a: [Float], b: [Float], aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    for i in 0 ..< aDims.0 {
        for j in 0 ..< bDims.1 {
            for k in 0 ..< aDims.1 {
                res[i+aDims.0*j] += a[i+aDims.0*k] * b[k+bDims.0*j]
            }
        }
    }
    return res
}

In [ ]:
// To try it out, we extract the scalars out of our MNIST data as an array.
let flatA = xTrain[0..<5].array.scalars
let flatB = weights.array.scalars
let (aDims,bDims) = ((5, 784), (784, 10))

In [ ]:
// Now that we've got everything together, we can try it out!
var resultArray = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)

In [ ]:
time(repeating: 100) {
    _ = swiftMatmul(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.11785614000000004 ms,   min: 0.110522 ms,   max: 0.163228 ms


Awesome, that is pretty fast - compare that to **835 ms** with Python!

You might be wondering what that `time(repeating:)` builtin is.  As you might guess, this is actually a Swift function - one that is using "trailing closure" syntax to specify the body of the timing block.  Trailing closures are passed as arguments to the function, and in this case, the function was defined in our 00_load_data workbook.  Let's take a look!


## Getting the performance and utility of C

This performance is pretty great, but we can do better.  Swift is a memory safe language (like Python), which means it has to do array bounds checks and some other stuff.  Fortunately, Swift is a pragmatic language that allows you to drop through this to get peak performance - check out Jeremy's article [High Performance Numeric Programming with Swift: Explorations and Reflections](https://www.fast.ai/2019/01/10/swift-numerics/) for a deep dive.

One thing you can do is use `UnsafePointer` (which is basically a raw C pointer) instead of using a bounds checked array.  This gives up safety, but gives us about a 2x speedup in this case!


In [ ]:
// a and b are the flattened array elements, aDims/bDims are the #rows/columns of the arrays.
func swiftMatmulUnsafe(a: UnsafePointer<Float>, b: UnsafePointer<Float>,
                       aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    res.withUnsafeMutableBufferPointer { res in 
        for i in 0 ..< aDims.0 {
            for j in 0 ..< bDims.1 {
                for k in 0 ..< aDims.1 {
                    res[i+aDims.0*j] += a[i+aDims.0*k] * b[k+bDims.0*j]
                }
            }
        }
    }
    return res
}
time(repeating: 100) {
    _ = swiftMatmulUnsafe(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.05980202999999999 ms,   min: 0.059207 ms,   max: 0.089812 ms


If you really want to fall down the rabbit hole, you can look at the [implementation of `UnsafePointer`](https://github.com/apple/swift/blob/tensorflow/stdlib/public/core/UnsafePointer.swift), which is of written in Swift wrapping LLVM pointer operations.  This means you can literally get the performance of C code directly in Swift, while providing easy to use high level APIs!

Swift even lets you transparently work with C APIs, just like it does with Python:


In [ ]:
import Glibc

let ptr : UnsafeMutableRawPointer = malloc(42)
print(type(of: ptr))
print("Uninitialized garbage!!!! ", ptr.load(as: UInt8.self))
free(ptr)

UnsafeMutableRawPointer
Uninitialized garbage!!!!  0


# Working with Tensor

In [ ]:
func TensorMatmul(_ a: Tensor<Float>, _ b: Tensor<Float>) -> Tensor<Float> {
    var res:Tensor<Float> = Tensor(repeating: 0.0, shape: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0]{
        for j in 0..<b.shape[1]{
            for k in 0..<a.shape[1]{
                res[i][j] += a[i][k] * b[k][j]
            }
        }
    }
    return res
}

In [ ]:
let m1 = Tensor<Float>(randomNormal: [5, 784])
let m2 = Tensor<Float>(randomNormal: [784, 10])

In [ ]:
time() { let res = TensorMatmul(m1, m2)}

average: 16905.78793 ms,   min: 16905.78793 ms,   max: 16905.78793 ms


In [ ]:
print(17000/0.059)

288135.593220339


Looping over `Tensor` indices is a bad idea! (For now - in the future this will be even faster than the Swift version above, and will be easier to write too.)

#### Elementwise ops

Operators (+,-,\*,/) are usually element-wise.

Examples of element-wise operations:

In [ ]:
var a = Tensor([10.0, 6, -4])
var b = Tensor([2.0, 8, 7])
(a,b)

▿ 2 elements
  - .0 : [10.0,  6.0, -4.0]
  - .1 : [2.0, 8.0, 7.0]


In [ ]:
a + b

[12.0, 14.0,  3.0]


Comparison operators (>,<,==,!=,...) are `true` if all the elements of the tensors satisfy the comparison. Elementwise versions have the . prefix: .>, .<, .== ...

In [ ]:
a < b

false


In [ ]:
a .< b

[false,  true,  true]


In [ ]:
var m = Tensor([1.0, 2, 3, 4, 5, 6, 7, 8, 9]).reshaped(to: [3,3])
m

[[1.0, 2.0, 3.0],
 [4.0, 5.0, 6.0],
 [7.0, 8.0, 9.0]]


In [ ]:
2 * m

[[ 2.0,  4.0,  6.0],
 [ 8.0, 10.0, 12.0],
 [14.0, 16.0, 18.0]]


In [ ]:
sqrt((m * m).sum().scalar!)

16.881943016134134


#### Elementwise matmul

In [ ]:
func elementWiseMatmul(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    let (ar,ac) = (a.shape[0],a.shape[1])
    let (br,bc) = (b.shape[0],b.shape[1])
    var res = Tensor<Float>(zeros: [ac, br])
    for i in 0..<ar {
        for j in 0..<bc {
            res[i][j] = (a[i] * b.slice(lowerBounds: [0,j], upperBounds: [ac,j+1]).squeezingShape(at: 1)).sum()
        }
    }
    return res
}

In [ ]:
let res = elementWiseMatmul(m1, m2)

In [ ]:
time() { let _ = elementWiseMatmul(m1, m2)}

average: 383.143201 ms,   min: 383.143201 ms,   max: 383.143201 ms


### Broadcasting

#### Broadcasting with a scalar

In [ ]:
var a = Tensor([10.0, 6.0, -4.0])

In [ ]:
print(a .> 0)

[ true,  true, false]


In [ ]:
print((a .> 0).all())

false


In [ ]:
print((a .> 0).any())

true


In Tensorflow the operator `>` between tensors will return `true` if all the elements of the first tensor are greater than the ones of the second tensor. `.>` makes the memberwise comparison.

In [ ]:
print(a+1)

[11.0,  7.0, -3.0]


In [ ]:
2 * m

[[ 2.0,  4.0,  6.0],
 [ 8.0, 10.0, 12.0],
 [14.0, 16.0, 18.0]]


#### Broadcasting a vector with a matrix

In [ ]:
let c = Tensor([10.0,20.0,30.0])

By default, broadcasting is done by adding 1 dimensions to the beginning until dimensions of both objects match.

In [ ]:
m + c

[[11.0, 22.0, 33.0],
 [14.0, 25.0, 36.0],
 [17.0, 28.0, 39.0]]


In [ ]:
c + m

[[11.0, 22.0, 33.0],
 [14.0, 25.0, 36.0],
 [17.0, 28.0, 39.0]]


To broadcast on the other dimensions, one has to use `expandingShape` to add the dimension.

In [ ]:
m + c.expandingShape(at: 1)

[[11.0, 12.0, 13.0],
 [24.0, 25.0, 26.0],
 [37.0, 38.0, 39.0]]


In [ ]:
c.expandingShape(at: 1)

[[10.0],
 [20.0],
 [30.0]]


#### Matmult with broadcasting

In [ ]:
func broadcastMatmult(_ a:Tensor<Float>, _ b:Tensor<Float>) -> Tensor<Float>{
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])
    for i in 0..<a.shape[0]{
        res[i] = (a[i].expandingShape(at: 1) * b).sum(squeezingAxes: 0)
    }
    return res
}

In [ ]:
let res = broadcastMatmult(m1, m2)

In [ ]:
time(repeating: 100) { let _ = broadcastMatmult(m1, m2)}

average: 2.3527557299999997 ms,   min: 2.243666 ms,   max: 2.526681 ms


#### Broadcasting rules

In [ ]:
c.expandingShape(at: 0).shape

▿ TensorShape
  ▿ dimensions : 2 elements
    - 0 : 1
    - 1 : 3


In [ ]:
c.expandingShape(at: 1).shape

▿ TensorShape
  ▿ dimensions : 2 elements
    - 0 : 3
    - 1 : 1


In [ ]:
c.expandingShape(at: 0) * c.expandingShape(at: 1)

[[100.0, 200.0, 300.0],
 [200.0, 400.0, 600.0],
 [300.0, 600.0, 900.0]]


In [ ]:
c.expandingShape(at: 0) .> c.expandingShape(at: 1)

[[false,  true,  true],
 [false, false,  true],
 [false, false, false]]


### Tensorflow op

In [ ]:
time(repeating: 100) { let _ = matmul(m1, m2)}

average: 0.024352040000000002 ms,   min: 0.023278 ms,   max: 0.053765 ms


Halide video snippet here - Jeremy.

### Export

In [ ]:
notebookToScript(fname: Path.cwd / "01_matmul.ipynb")